<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [4]:
!pip install gensim

In [3]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2025-10-06 15:34:09--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2025-10-06 15:34:10--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1.2’

SO_vectors_200.bin? 100%[===================>]   1.35G  8.66MB/s    in 3m 2s   

2025-10-06 15:37:12 (7.61 MB/s) - ‘SO_vectors_200.bin?download=1.2’ saved [1453905423/1453905423]



In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1.1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Проверка как работает используемый метод из документации
def most_similar(threshold=0.6):

    similar_words = []

    # check docs for this method
    top_words = wv_embeddings.most_similar('dog', topn=5)

    for word, similarity in top_words:
        if similarity > threshold:
            similar_words.append((word, similarity))

    return similar_words

sim_words = most_similar()
sim_words

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

In [5]:
def check_cat_position():

    similar_words = wv_embeddings.most_similar('dog', topn=100)

    # Search cat
    cat_position = None
    for i, (word, similarity) in enumerate(similar_words, 1):
        if word == 'cat':
            cat_position = i
            break

    in_top_5 = cat_position is not None and cat_position <= 5
    return in_top_5, cat_position

print("Позиция слова cat :", check_cat_position()[1])
print("ОТВЕТ:")
print("'cat' не входит в топ-5 ближайших слов к 'dog'.") if check_cat_position()[1] else "CAT входит в топ 5 ближайщих"

Позиция слова cat : 26
ОТВЕТ:
'cat' не входит в топ-5 ближайших слов к 'dog'.


***Ваш ответ:*** 'cat' не входит в топ-5 ближайших слов к 'dog'.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

"""class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

tokenizer = MyTokenizer()"""

<>:15: SyntaxWarning: invalid escape sequence '\w'
<>:15: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-3848683184.py:15: SyntaxWarning: invalid escape sequence '\w'
  return re.findall('\w+', text)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


"class MyTokenizer:\n    def __init__(self):\n        pass\n    def tokenize(self, text):\n        return re.findall('\\w+', text)\n\ntokenizer = MyTokenizer()"

In [7]:

def simple_tokenize(text):
    #return re.findall(r'\w+', text)
    return tokenizer(text)

def question_to_vec(question, embeddings, dim=None):
    if dim is None:
        dim = embeddings.vector_size
    tokens = [token.lower() for token in simple_tokenize(question)]
    valid_vectors = []
    for token in tokens:
        if token in embeddings:
            valid_vectors.append(embeddings[token])
    if valid_vectors:
        return np.mean(valid_vectors, axis=0)
    else:
        return np.zeros(dim)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [8]:
wv_embeddings['love']

array([-1.1766732 , -2.050077  , -0.898872  , -0.07775963, -0.1924933 ,
       -0.8429467 , -1.0288097 , -3.174912  ,  0.5573096 , -0.12763615,
        0.49262068,  3.9925249 ,  0.27934542, -0.8876889 , -0.1980355 ,
       -0.629465  ,  0.8875056 ,  1.7594627 ,  1.1172884 ,  2.9088626 ,
        1.3585839 ,  2.6186543 , -1.6687378 , -0.85575104,  0.78545505,
       -0.5778112 ,  0.8198755 , -0.00422503,  0.78299004, -0.12864277,
       -1.7777488 , -3.1062195 , -0.8176724 , -1.0965716 ,  1.6746812 ,
        1.421553  , -0.45011058,  0.8663928 , -0.09573402, -0.20228954,
        0.97701675, -0.4227626 , -0.67955714,  1.1165454 , -0.07940765,
        1.352443  ,  1.2797256 , -0.547621  , -1.3387707 , -0.16459484,
       -0.966921  ,  0.6784198 , -0.5505769 ,  0.8784912 , -0.80668056,
        1.8429828 ,  2.3197317 , -0.3311743 ,  0.9486556 , -1.7003541 ,
       -0.3480606 , -2.088733  ,  1.1039977 ,  0.28361505,  0.5159701 ,
        1.3285043 , -0.9965045 , -0.4221305 ,  0.49064258, -1.40

Это означает, что в вашей модели SO_vectors_200.bin найдено хотя бы одно слово из этого предложения, и среднее векторов дало именно такое значение

In [9]:
vec = question_to_vec("I love neural networks", wv_embeddings)

result = round(vec[2], 2)
result

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)



#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [10]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k
    if not dup_ranks:
        return 0.0
    hits = sum(1 for rank in dup_ranks if rank <= k)
    return hits / len(dup_ranks)

In [11]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [12]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    if not dup_ranks:
        return 0.0

    total = 0.0
    for rank in dup_ranks:
        if rank <= k:
            total += 1.0 / math.log2(1 + rank)

    dcg_value = total / len(dup_ranks)
    return dcg_value

In [13]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
import pandas as pd

In [15]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


Решение:

Дубликат находится на 2-й позиции (1-based), то есть dup_ranks = [2].

Теперь вычислим метрики:

Hits@k для k = 1, 2, 3, 4 = [0.0, 1.0, 1.0, 1.0]
DCG@k = (1 / log₂(1 + 2)) / 1 = 1 / log₂(3) ≈ 1 / 1.58496 ≈ 0.63093

сли k ≥ 2! При k=1, дубликат не входит то 0
k=1: 0
k=2,3,4: ≈ 0.63093

У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
replace data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/.DS_Store          
replace __MACOSX/data/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/data/._.DS_Store  
replace data/train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/train.tsv          
replace data/validation.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data/validation.tsv     


Считайте данные.

In [18]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [19]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [20]:
len(validation_data)

3760

Размер нескольких первых строк

In [21]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [23]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
    question: строка
    candidates: массив строк (кандидатов) [a, b, c]
    embeddings: словарь {токен: np.array(размерности dim)}
    tokenizer: функция, преобразующая строку в список токенов
    dim: размерность векторов слов
    result: список пар (начальная позиция, кандидат), отсортированный по убыванию сходства
    """

    def get_sentence_embedding(sentence):
        return question_to_vec(sentence, embeddings, dim)

    q_emb = get_sentence_embedding(question).reshape(1, -1)  # shape (1, dim)

    cand_embs = []
    for cand in candidates:
        emb = get_sentence_embedding(cand)
        cand_embs.append(emb)
    cand_embs = np.array(cand_embs)  # shape (n_candidates, dim)

    similarities = cosine_similarity(q_emb, cand_embs).flatten()  # shape (n_candidates,)
    # Создаём список (сходство, исходный индекс, кандидат)
    scored = [(similarities[i], i, candidates[i]) for i in range(len(candidates))]
    scored.sort(key=lambda x: x[0], reverse=True)

    result = [(idx, cand) for (_, idx, cand) in scored]
    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [24]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [25]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, simple_tokenize())
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [26]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
            [('*', 'Getting all list items of an unordered list in PHP'),
            ('*', 'select2 not displaying search results'),
            ('*', 'WPF- How to update the changes in list item of a list')]]

In [27]:
results

[[(1, 'C# create cookie from string and send it'),
  (0, 'Convert Google results object (pure js) to Python object'),
  (2, 'How to use jQuery AJAX for an outside domain?')],
 [('*', 'Getting all list items of an unordered list in PHP'),
  ('*', 'select2 not displaying search results'),
  ('*', 'WPF- How to update the changes in list item of a list')]]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [28]:
from tqdm.notebook import tqdm

In [29]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [30]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.411 | Hits@   1: 0.411
DCG@   5: 0.500 | Hits@   5: 0.579
DCG@  10: 0.525 | Hits@  10: 0.656
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.976
DCG@1000: 0.586 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

In [34]:
question = 'Sending array via Ajax fails'
candidates = [
    'Getting all list items of an unordered list in PHP',
    'WPF- How to update the changes in list item of a list',
    'select2 not displaying search results'
]

ranks = rank_candidates(question, candidates, wv_embeddings, tokenizer)
print("Последовательность индексов:", ''.join(str(r[0]) for r in ranks))

Последовательность индексов: 021


Ответ:
Hits@1: 0.411 - только в 41.1% случаев правильный ответ на 1-м месте

Hits@5: 0.579 - в 57.9% случаев правильный ответ в топ-5

Hits@1000: 1.000 - на всей выборке (1000 кандидатов) всегда находим правильный ответ

Последовательность: 021

### Эмбеддинги, обученные на корпусе похожих вопросов

In [35]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [36]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [41]:
from tqdm import tqdm
import gensim

corpus = []

for line in tqdm(train_data):
    if len(line) < 2:
        continue

    question = line[0]
    candidates = line[1:]

    for candidate in candidates:
        combined_text = question + " " + candidate
        tokens = simple_tokenize(combined_text)

        if len(tokens) >= 3:
            corpus.append(tokens)

100%|██████████| 1000000/1000000 [00:04<00:00, 205755.72it/s]


In [42]:
from gensim.models import Word2Vec

vector_size = 200
min_count = 1  # так как у нас специфические технические термины из стака

# Выбираем window = 8, потому что:
# Вопросы на Stack часто содержат сложные технические формулировки
# Между терминами в технических вопросах могут быть длинные зависимости
# window=8 позволяет захватить семантические связи между удаленными словами


model = gensim.models.Word2Vec(
    corpus,
    vector_size=vector_size,
    window=8,
    min_count=min_count,
    workers=4,
    epochs=10
)

wv_embeddings = model.wv

In [43]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 3760/3760 [10:39<00:00,  5.88it/s]


In [44]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

100%|██████████| 6/6 [00:00<00:00, 3010.99it/s]

DCG@   1: 0.294 | Hits@   1: 0.294
DCG@   5: 0.374 | Hits@   5: 0.447
DCG@  10: 0.392 | Hits@  10: 0.505
DCG@ 100: 0.442 | Hits@ 100: 0.752
DCG@ 500: 0.465 | Hits@ 500: 0.929
DCG@1000: 0.472 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:



Качество модели достаточно низкое:

- Hits@5: 0.447 - менее чем в половине случаев правильный ответ в топ-5

Модель работает лучше на больших K, но для практического применения это бесполезно

Анализ проблем:

Принцип токенизации:

- Qwen tokenizer (предназначен для китайского и английского) может плохо обрабатывать технические термины программирования

- Лучше использовать специализированную токенизацию для кода: сохранение camelCase, snake_case, специальных символов

Нормализация слов:

- Приведение к нижнему регистру помогает, но может терять важную информацию (Python vs python)

- Для технических текстов лучше частичная нормализация - сохранять регистр для идентификаторов

Качество эмбеддингов:

- Word2Vec на маленьком корпусе плохо улавливает семантику технических терминов

- Лучше использовать предобученные эмбеддинги на большом корпусе технических текстов

Причины плохого качества:
- Недостаточный объем данных для обучения
- Плохая токенизация технических терминов
- Простое усреднение векторов не отражает семантику сложных вопросов
- Отсутствие учета синтаксической структуры

Предлагаемый подход:

- Использовать предобученные эмбеддинги (FastText, GloVe) + дообучение
- Специализированная токенизация для кода:
    - Сохранять camelCase (разбивать на слова)
    - Сохранять специальные символы
- Использовать BERT-like модели для получения контекстных эмбеддингов
- Применять взвешенное усреднение (TF-IDF) вместо простого среднего

Я предлагаю применить еще следующее:

- Использовать Sentence-BERT для получения эмбеддингов целых предложений

- Применить transfer learning с предобученными моделями

- Добавить features: длина вопроса, общие слова, синтаксическое сходство

- Использовать siamese neural networks для пар вопросов